# Interpolacja 

### Zadanie 1
Stablicuj następujące funkcje: sqrt(x), sin(x), x^3+2x w czterech punktach należących do przedziału 0 do 10.

In [2]:
import numpy as np
def lookupTab(fx, steps):
  val_x = np.array(np.linspace(0, 10, steps))
  array_y = np.zeros([steps])
  for i in range(len(val_x)):
    array_y[i] = fx(val_x[i])
  return array_y

lookup_sine = lookupTab(lambda x : np.sin(x), 4)
print(lookup_sine)

lookup_sqrt = lookupTab(lambda x : np.sqrt(x), 4)
print(lookup_sqrt)


lookup_func3 = lookupTab(lambda x : x**3+2*x, 4)
print(lookup_func3)

[ 0.         -0.19056796  0.37415123 -0.54402111]
[0.         1.82574186 2.5819889  3.16227766]
[   0.           43.7037037   309.62962963 1020.        ]


### Zadanie 2 
Napisz algorytm znajdujący wielomian interpolujący Lagrange dla powyższych stablicowanych funkcji.

In [20]:

import scipy
from scipy.interpolate import lagrange
import numpy as np


def lagrangePoly(val_y):
    val_x  = np.array(np.linspace(0, 10, len(val_y)))
    n = len(val_y)
    poly = np.zeros(n)
    def basis(i):    #dodatkowa fcja obliczająca mianownik
      p = 1
      for j in (j for j in range(0, n) if (j != i)):
           p *= (val_x[i] - val_x[j])
      return p
    for i in range(len(val_y)):
        prod = 1
        prod = val_y[i] / basis(i)
        exp = [prod] + (n-1)*[0]
        for j in (j for j in range(n) if (j != i)):
            for k in range(n-1,0,-1):
                exp[k] += exp[k-1]
                exp[k-1] *= (-val_x[j])
        for j in range(n):
          poly[j] += exp[j]
    return(np.around(poly, decimals =20))



val_x  = np.array(np.linspace(0, 10, len(val_y)))
print("Lagrange polynomial for: \n")
print("f(x) = sin(x)\n")
val_y = lookupTab(lambda x : np.sin(x), 4)
print("my algorithm: ",lagrangePoly(val_y)[::-1])
print("\nscimpy func: ",scipy.interpolate.lagrange(val_x, val_y))
print("\n ===================================")
print("\nLagrange polynomial for: \n")
print("f(x) = sqrt(x)\n")
val_y = lookupTab(lambda x : np.sqrt(x), 4)
print("my algorithm: ",lagrangePoly(val_y)[::-1])
print("\nscimpy func: ",scipy.interpolate.lagrange(val_x, val_y))
print("\n ===================================")
print("\nLagrange polynomial for: \n")
print("f(x) =  x**3+2*x\n")
val_y = lookupTab(lambda x :  (x**3+2*x), 4)
print("my algorithm: ",lagrangePoly(val_y)[::-1])                # wartości współczynników dla ostatniej funkcji są bardzo małe, a zaimplementowany przeze mnie algorytm nie ma wystarczającej dokładności
print("\nscimpy func: ",scipy.interpolate.lagrange(val_x, val_y)) #z czego wynikają różnice między wynikiem otrzymanym z tej fcji a wynikiem fcji z scimpy

Lagrange polynomial for: 

f(x) = sin(x)

my algorithm:  [-0.0100718   0.13470596 -0.39428133  0.        ]

scimpy func:            3          2
-0.01007 x + 0.1347 x - 0.3943 x


Lagrange polynomial for: 

f(x) = sqrt(x)

my algorithm:  [ 0.00402091 -0.08833641  0.79750043  0.        ]

scimpy func:            3           2
0.004021 x - 0.08834 x + 0.7975 x


Lagrange polynomial for: 

f(x) =  x**3+2*x

my algorithm:  [1. 0. 2. 0.]

scimpy func:     3             2
1 x - 7.105e-15 x + 2 x


### Zadanie 3
Porównaj wartość dokładną z wynikiem interpolacji dla punktów znajdujących się pomiędzy węzłami wielomianu (w połowie odległości) interpolującego. Oszacuj dokładność interpolacji. 


In [27]:
import statistics

def getPoins(steps):
    val_x=np.linspace(0, 10, steps)
    points=np.zeros([len(val_x)-1])
    for i in range(len(val_x)-1):
        points[i]=(val_x[i+1]-val_x[i])/2+val_x[i]
    return points

def interpolatePoints(fx, steps):
  points = getPoins(steps)
  results = np.zeros(len(points))
  for i in range(len(points)):
    polynomial = lagrangePoly(lookupTab(fx, steps))
    for j in range(0, len(polynomial)):
      results[i]+=polynomial[j]*(points[i]**j)

  return results


def ComparePrecision(fx, steps):
  interpolationResults = interpolatePoints(fx, steps)
  points = getPoins(steps)
  print("results of interpolation:  ", interpolationResults)
  print("accurate results:  ", fx(points))
  diff = np.zeros(len(points))
  for i in range(0, len(fx(points))):
    diff[i] = abs(interpolationResults[i] - fx(points)[i])
  print("variance:  ",statistics.variance(diff))
  print("median:  ", statistics.median(diff))
  print("standard deviation: ",statistics.stdev(diff))


print("sin()x \n")
ComparePrecision(lambda x: np.sin(x), 4)


print("\n \n \nsqrt()x \n")
ComparePrecision(lambda x: np.sqrt(x), 4)

print("\n \n \nx**3+2*x \n")
ComparePrecision(lambda x: (x**3+2*x), 4)

sin()x 

results of interpolation:   [-0.32958104  0.13726691  0.24031267]
accurate results:   [ 0.99540796 -0.95892427  0.88729411]
variance:   0.11897201548493072
median:   1.096191182172574
standard deviation:  0.3449232023000638

 
 
sqrt()x 

results of interpolation:   [1.10240382 2.2817062  2.83828203]
accurate results:   [1.29099445 2.23606798 2.88675135]
variance:   0.006679565213717396
median:   0.048469316500423965
standard deviation:  0.0817286070707032

 
 
x**3+2*x 

results of interpolation:   [  7.96296296 135.         595.37037037]
accurate results:   [  7.96296296 135.         595.37037037]
variance:   2.059715823532062e-27
median:   5.684341886080802e-14
standard deviation:  4.538409218583161e-14


### Zadanie 4
Powtórz powyższe kroki dla 3, 5 i 8 węzłów interpolacji - podsumuj badania. 

In [23]:

  
print("3 steps\n\n")
print("sin()x \n")
ComparePrecision(lambda x: np.sin(x), 3)


print("\n \n \nsqrt()x \n")
ComparePrecision(lambda x: np.sqrt(x), 3)

print("\n \n \nx**3+2*x \n")
ComparePrecision(lambda x: (x**3+2*x), 3)

print("\n========================================")
print("\n\n5 steps\n\n")
print("sin()x \n")
ComparePrecision(lambda x: np.sin(x), 5)


print("\n \n \nsqrt()x \n")
ComparePrecision(lambda x: np.sqrt(x), 5)

print("\n \n \nx**3+2*x \n")
ComparePrecision(lambda x: (x**3+2*x),5)
print("\n========================================")
print("\n\n8 steps\n\n")
print("sin()x \n")
ComparePrecision(lambda x: np.sin(x), 8)


print("\n \n \nsqrt()x \n")
ComparePrecision(lambda x: np.sqrt(x), 8)

print("\n \n \nx**3+2*x \n")
ComparePrecision(lambda x: (x**3+2*x),8)


print("po przeanalizowaniu wyników dla kolejno 3, 5 i 8 węzłów interpolacji, wnioskujemy że wynik interpolacji jest tym dokładniejszy, im więcej węzłów rozpatrzymy")


3 steps


sin()x 

results of interpolation:   [-0.65119057 -0.92320112]
accurate results:   [0.59847214 0.93799998]
variance:   0.1869896000694319
median:   1.5554319052978727
standard deviation:  0.43242294119233765

 
 
sqrt()x 

results of interpolation:   [1.28176628 2.86290511]
accurate results:   [1.58113883 2.73861279]
variance:   0.015326544574625576
median:   0.21183243632127458
standard deviation:  0.12380042235237154

 
 
x**3+2*x 

results of interpolation:   [-26.25 483.75]
accurate results:   [ 20.625 436.875]
variance:   8.17890986533145e-27
median:   46.87499999999995
standard deviation:  9.043732562018544e-14



5 steps


sin()x 

results of interpolation:   [ 1.40542894 -0.5530228  -0.30681659  1.5325092 ]
accurate results:   [ 0.94898462 -0.57156132 -0.03317922  0.62472395]
variance:   0.1405722662103963
median:   0.3650408466810219
standard deviation:  0.3749296816876417

 
 
sqrt()x 

results of interpolation:   [0.982066   1.95960176 2.48538063 2.98306748]
accura